# Notebook to perform perfomance analysis

Performance data will be saved on csv files

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('log.csv')
df['phase'] = df.replace({'phase': {0: 'train', 1: 'val', 2: 'test'}})['phase']
df.head(n=10)

,epoch,batch,batch_time(s),phase
0,0,0,1.655002,train
1,0,1,1.181834,train
2,0,2,1.947835,train
3,0,3,0.049874,train
4,0,0,0.110931,val
5,0,1,1.584433,val
6,0,2,0.080650,val
7,0,3,0.015364,val
8,1,0,0.193496,train
9,1,1,0.183684,train
